In [1]:
import os, shutil
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from IPython.display import Image
import imutils
import matplotlib.image as mpimg
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import zipfile
z = zipfile.ZipFile("archive.zip")
z.extractall()

In [3]:
folder = 'brain_tumor_dataset/yes/'
count = 1

for filename in os.listdir(folder):
    source = folder + filename
    destination = folder + "Y_"+str(count)+".jpg"
    os.rename(source,destination)
    count+=1
    
print("All Files are renamed in the yes dir")

All Files are renamed in the yes dir


In [4]:
folder = 'brain_tumor_dataset/no/'
count = 1

for filename in os.listdir(folder):
    source = folder + filename
    destination = folder + "N_"+str(count)+".jpg"
    os.rename(source,destination)
    count+=1
    
print("All Files are renamed in the no dir")

All Files are renamed in the no dir
